# 🍎 파이썬 머신러닝 완벽 가이드 혼공

### 2019.05.27 ~ 2019.06.02 교재 09장

### 09. 추천 시스템

#### 08. 파이썬 추천 시스템 패키지 - Surprise 

파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 Surprise 

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy 
from surprise.model_selection import train_test_split 

data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [2]:
algo = SVD()
algo.fit(trainset)

- test(): **사용자-아이템 평점 데이터 세트 전체**에 대해서 추천을 예측
- predict(): **개별 사용자와 영화**에 대한 추천 평점을 반환

In [3]:
# test()로 예측하기 
predictions = algo.test(testset)
print(type(predictions), len(predictions))
predictions[:5]

<class 'list'> 25000


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.7944274832016704, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.5544093095251603, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.458045425043873, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.2063142279447616, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.493229505858857, details={'was_impossible': False})]

- uid: 사용자, iid=영화 아이디, r_ui=실제 예측값, est=추천예측평점

In [4]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.7944274832016704),
 ('882', '291', 3.5544093095251603),
 ('535', '507', 4.458045425043873)]

In [5]:
# predict()로 예측하기 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.228671685773864, details={'was_impossible': False})

테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이를 평가해보기

- Surprise의 accuracy 모듈: RMSE, MSE 등의 방법

In [6]:
accuracy.rmse(predictions)

RMSE: 0.9458


0.9457864473020249

##### Surprise 주요 모듈 소개 

- user_id, item-id, rating 데이터가 로우 레벨로 된 데이터 세트만 적용
- 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있으면 안됨.

In [7]:
import pandas as pd 

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
# header와 index가 삭제된 파일 저장 
ratings.to_csv('data/ml-latest-small/ratings_noh.csv', 
              index=False, header=False)

In [8]:
from surprise import Reader 

# 읽기 전에 데이터의 파싱 포맷을 정의해야 함! 
# 평점의 단위는 0.5, 최대 평점은 5
reader = Reader(line_format='user item rating timestamp', sep=',',
               rating_scale=(0.5, 5))
data = Dataset.load_from_file('data/ml-latest-small/ratings_noh.csv',
                             reader=reader)

In [9]:
# SVD 행렬 분해 기법을 이용한 추천 예측 
trainset, testset = train_test_split(data, test_size=.25,
                                    random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

##### 판다스 데이터프레임에서 Surprise 데이터 세트로 로딩 
- 데이터프레임 또한 사용자 아이디, 아이템 아이디, 평점 칼럼 순서를 지켜야 함

In [11]:
from surprise import Reader, Dataset

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId','rating']], reader)
trainset, testset = train_test_split(data, 
                                    test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

##### 베이스라인 평점 

어떤 사람은 평가에 후하고, 어떤 사람은 깐깐할 수 있음. 이러한 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것을 베이스라인 평점이라고 한다. 

- 베이스라인 평점 = 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 

##### 교차 검증과 하이퍼 파라미터 튜닝 
- 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate()와 GridSearchCV 클래스를 제공함. 
- cross_validate()를 이용해 ratings.csv를 데이터프레임으로 로딩한 데이터를 5개의 학습/검증 폴드 데이터 세트로 분리해 교차 검증을 수행하고 RMSE, MAE로 성능 평가를 진행함. 

In [13]:
from surprise.model_selection import cross_validate 

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 
                                    'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], 
              cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8785  0.8777  0.8698  0.8737  0.8738  0.8747  0.0032  
MAE (testset)     0.6750  0.6731  0.6698  0.6704  0.6705  0.6717  0.0020  
Fit time          4.25    4.57    4.42    4.24    4.27    4.35    0.13    
Test time         0.13    0.19    0.19    0.13    0.19    0.16    0.03    


{'test_rmse': array([0.87849156, 0.87774696, 0.86978169, 0.87367154, 0.87379494]),
 'test_mae': array([0.67500684, 0.67308242, 0.66976263, 0.67043437, 0.67045695]),
 'fit_time': (4.252317905426025,
  4.573483943939209,
  4.418211221694946,
  4.2431321144104,
  4.265954256057739),
 'test_time': (0.12540817260742188,
  0.19124722480773926,
  0.1925210952758789,
  0.12508583068847656,
  0.18961310386657715)}

In [14]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정 
param_grid = {'n_epochs': [20,40,60], 
             'n_factors': [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'],cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터 
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8769986072643192
{'n_epochs': 20, 'n_factors': 50}


##### Surprise를 이용한 개인화 영화 추천 시스템 구축 

Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수 없음.

In [15]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

이렇게 오류가 뜬다는 말씀. 그래서 학습 데이터로 사용하기위해 DatasetAutoFolds 클래스를 이용하면 된다.

In [16]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp',
               sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='data/ml-latest-small/ratings_noh.csv',
                             reader=reader)

# 전체 데이터를 학습 데이터로 생성함.
trainset = data_folds.build_full_trainset()

In [17]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [19]:
# 영화에 대한 상세 속성 정보 데이터프레임 로딩
movies = pd.read_csv('data/ml-latest-small/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds == 42].count() ==0: 
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음 ')
    
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음 
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [20]:
# 사용자 아이디 9의 무비 아이디 42 평점 예측 
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [21]:
#전체 데이터 예측하기 
def get_unseen_surprise(ratings, movies, userId): 
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수: ', 
         len(unseen_movies), '전체 영화 수:', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수:  9696 전체 영화 수: 9742


In [24]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) 
                  for movieId in unseen_movies]
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions=predictions[:top_n]
    
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(id, title, rating) for id, title, rating 
                      in zip(top_movie_ids, top_movie_titles, 
                            top_movie_rating)]
    
    return top_movie_preds

In [25]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수:  9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122758
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
